# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [2]:
%pip install causal_conv1d datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:

# Lab

In [3]:
import torch, InitMamba, importlib
importlib.reload(InitMamba)
import VAE
importlib.reload(VAE)
from VAE import MambaVAE
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer.add_special_tokens({"bos_token": "<|startoftext|>"})
vae = MambaVAE().cuda().eval()
vae.load_state_dict(torch.load('./results/CoT_vae4/model.pth', weights_only=True))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/517M [00:00<?, ?B/s]

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

<All keys matched successfully>

In [5]:
inputs1 = tokenizer(["<|startoftext|>According to the given context, as temperature increases particles move faster. Tina's body is at a colder temperature than before, so we can infer that there will be less energy in her body and thus less movement of particles. So slower would be the answer because it matches with our inference from the temperatures mentioned above Answer: slower<|endoftext|>"], return_tensors='pt').to('cuda')
inputs2 = tokenizer(['<|startoftext|>The word "sad" is repeated multiple times in the text and it is also combined with other words indicating sadness like "late", "birthday". Hence, it can be inferred that the underlying emotion of the text is sad. Answer: sad<|endoftext|>'], return_tensors='pt').to('cuda')

In [6]:
res1 = vae(**inputs1)
print(res1[:2])
print(tokenizer.batch_decode(res1[2].argmax(-1)))

res2 = vae(**inputs2)
print(res2[:2])
print(tokenizer.batch_decode(res2[2].argmax(-1)))

The 'batch_size' argument of MambaCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.


(tensor(0.6392, device='cuda:0', grad_fn=<DivBackward0>), tensor(0., device='cuda:0'))
["According to the given context, as temperature increases particles of faster.\n's body is at a higherder temperature than before, so she can infer that there will be less energy in her body and thus less movement of particles. So the would be the answer here it matches with what reasoning from the context mentioned above.: slower<|endoftext|><|endoftext|>"]
(tensor(0.3894, device='cuda:0', grad_fn=<DivBackward0>), tensor(0., device='cuda:0'))
['The word "sad" is a multiple times in the text and it is also used with other words. sadness. "late", "sday", Hence, the can be inferred that the underlying emotion of the text is sad. Answer: sad<|endoftext|><|endoftext|>']


In [10]:
r = 0.5
noise = torch.randn_like(res1[3])
mix_state = (res1[3]*r+noise*(1-r))
tokens = vae.decoder.generate(**tokenizer('<|startoftext|>', return_tensors='pt').to('cuda'),
                              inputs_ssm_states = mix_state,
                              max_length = 128,
                              # do_sample=True,
                              # num_return_sequences=3
                              )
tokenizer.batch_decode(tokens, skip_special_tokens=True)

['The answer is "The man is a man in a red and white shirt, a red and white shirt, and a red and white shirt." because the question asks for information about a man. Answer: The man is a man in a red and white shirt, a red and white shirt, and a red and white shirt.']

# Eval


In [ ]:
import torch, transformers, os
from transformers import TrainingArguments, AutoTokenizer
from InitMamba import MambaForCausalLM
from datasets import load_dataset
import dataset.Dataset as Dataset

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
model = MambaForCausalLM.from_pretrained('state-spaces/mamba-130m-hf').cuda()
dataset = load_dataset("arrow", data_files = './cache/CoT_full.arrow', split = 'train').select(range(100000))
print(dataset)
tot = len(dataset)
eval_size= int(tot * 0.05)
train_dataset = dataset.select(range(eval_size, tot))
eval_dataset = dataset.select(range(eval_size))
model.load_state_dict(torch.load("results/CoT_full/model.pth", weights_only=True))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


Generating train split: 0 examples [00:00, ? examples/s]

KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
input_str = tokenizer.decode(eval_dataset[1]['input_ids'], skip_special_tokens=True)
input_ids = tokenizer(input_str.split(' Thinking: ')[0] + ' Thinking: ', return_tensors = 'pt')['input_ids'].cuda()
tokens = model.generate(
    input_ids,
    max_length = 512,
    # num_return_sequences = 5,
    # do_sample = False,
    # num_beams = 5
    )
print(input_str)
tokenizer.batch_decode(tokens)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The 'batch_size' argument of MambaCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.


Choose your reply from the options at the end. Does "The Tucson-Pima County Bicycle Advisory Committee (TPCBAC) serves in an advisory capacity to local governments on issues relating to bicycle recreation, transportation, and safety." contain the correct answer to "What organization advises the Tucson government on bike concerns?"
OPTIONS:
- yes
- no Thinking: The passage talks about the Tucson-Pima County Bicycle Advisory Committee serving in an advisory capacity to local governments on issues relating to bicycle recreation, transportation and safety. So, it does contain the correct answer for what organization advises the Tucson government on bike concerns. Hence, yes is our choice of reply Answer: yes


['Choose your reply from the options at the end. Does "The Tucson-Pima County Bicycle Advisory Committee (TPCBAC) serves in an advisory capacity to local governments on issues relating to bicycle recreation, transportation, and safety." contain the correct answer to "What organization advises the Tucson government on bike concerns?"\nOPTIONS:\n- yes\n- no Thinking: \nThe passage talks about the Tucson-Pima County Bicycle Advisory Committee (TPCBAC) serving in an advisory capacity to local governments on issues relating to bicycle recreation, transportation and safety. It does not mention anything about bike concerns or whether it advises the government on these issues. So, the answer is "no". Answer: no<|endoftext|>']

# Run

In [ ]:
!python VAE_trainer.py

tokenizer_config.json: 100% 4.79k/4.79k [00:00<00:00, 29.4MB/s]
tokenizer.json: 100% 2.11M/2.11M [00:00<00:00, 28.0MB/s]
E0000 00:00:1739913489.035583    1912 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739913489.042507    1912 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Generating train split: 1466151 examples [00:09, 147241.63 examples/s]
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1466151
})
config.json: 100% 895/895 [00:00<00:00, 7.32MB/s]
model.safetensors: 100% 517M/517M [00:12<00:00, 41.6MB/s]
The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow ht

In [ ]:
!python data/Dataset.py

In [ ]:
!python trainer/CoT_full_trainer.py

In [ ]:
from google.colab import runtime
runtime.unassign()